In [1]:
import torch
from torch_geometric.data import Data

# 示例1：构造第一个图（晶胞1）
# 节点特征：3个节点，每个节点有6维特征
node_features1 = torch.tensor([
    [1.0, 2.0, 3.0, 4.0, 5.0, 6.0],   # 节点0的特征
    [2.0, 3.0, 4.0, 5.0, 6.0, 7.0],   # 节点1的特征
    [3.0, 4.0, 5.0, 6.0, 7.0, 8.0]    # 节点2的特征
], dtype=torch.float)

# 邻接关系：无向图，用边索引表示 (这里构造一个链式连接: 0-1-2)
edge_index1 = torch.tensor([
    [0, 1, 1, 2],  # 源节点
    [1, 0, 2, 1]   # 目标节点
], dtype=torch.long)

# 图的目标值（例如某种物理性质），单一浮点数
target1 = torch.tensor([0.5], dtype=torch.float)

# 使用 PyG 的 Data 对象封装图1的数据
graph1 = Data(x=node_features1, edge_index=edge_index1, y=target1)

# 示例2：构造第二个图（晶胞2）
# 节点特征：4个节点，每个节点6维特征
node_features2 = torch.tensor([
    [1.0, 0.0, 0.0, 1.0, 0.0, 1.0],  # 节点0的特征
    [0.0, 1.0, 0.0, 1.0, 1.0, 0.0],  # 节点1的特征
    [0.0, 0.0, 1.0, 0.0, 1.0, 1.0],  # 节点2的特征
    [1.0, 1.0, 1.0, 1.0, 0.0, 0.0]   # 节点3的特征
], dtype=torch.float)

# 邻接关系：构造一个链式连接: 0-1-2-3
edge_index2 = torch.tensor([
    [0, 1, 1, 2, 2, 3],  # 源节点
    [1, 0, 2, 1, 3, 2]   # 目标节点
], dtype=torch.long)

# 图的目标值
target2 = torch.tensor([1.0], dtype=torch.float)

# 封装图2的数据
graph2 = Data(x=node_features2, edge_index=edge_index2, y=target2)


In [2]:
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool

# 定义GCN模型用于图回归
class GCNNet(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(GCNNet, self).__init__()
        # 定义两层GCN卷积层
        self.conv1 = GCNConv(input_dim, hidden_dim)   # 第一层，将节点特征维度从input_dim升维到hidden_dim
        self.conv2 = GCNConv(hidden_dim, hidden_dim)  # 第二层
        self.conv3 = GCNConv(hidden_dim, hidden_dim)
        # 定义线性层用于最终回归输出
        self.fc = nn.Linear(hidden_dim, 1)            # 将图的隐藏表示映射为一个标量

    def forward(self, data):
        # 从data中获取信息，若batch不存在则创建
        x, edge_index = data.x, data.edge_index
        batch = data.batch if hasattr(data, 'batch') else None

        # 第一层GCN卷积和激活
        x = self.conv1(x, edge_index)   # 应用第一层GCN卷积
        x = F.relu(x)                   # ReLU激活增加非线性
        # 第二层GCN卷积和激活
        x = self.conv2(x, edge_index)   # 应用第二层GCN卷积
        x = F.relu(x)                   # ReLU激活

        # 如果batch为空（单图），则构造batch张量（将所有节点视为同一图的索引0）
        if batch is None:
            batch = x.new_zeros(x.size(0), dtype=torch.long)
        # 全局平均池化，将每个图的所有节点特征取平均，得到图级别特征
        x = global_mean_pool(x, batch)  # 结果维度: [num_graphs, hidden_dim]

        # 全连接层输出单一值，使用 squeeze 去除多余的维度
        out = self.fc(x)                # 结果维度: [num_graphs, 1]
        out = out.view(-1)              # 转换为形状 [num_graphs] 的向量
        return out


In [3]:
model = GCNNet(input_dim=6, hidden_dim=16)
with torch.no_grad():
    test = model(graph1)
    
test

tensor([-1.3542])

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [14]:
from torch_geometric.loader import DataLoader

# 准备数据集和数据加载器
dataset = [graph1.to(device), graph2.to(device)] * 10000                     # 我们的图数据列表

In [15]:
loader = DataLoader(dataset, batch_size=2)      # 每次加载2个图（这里正好全部数据一起）

# 初始化模型、损失函数和优化器
model = GCNNet(input_dim=6, hidden_dim=16)      # 输入维度6，对应节点特征长度；隐藏维度设为16
criterion = nn.MSELoss()                        # 均方误差损失函数
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# 训练模型
model.train()                                   # 切换模型到训练模式
epochs = 100

In [16]:
from tqdm.notebook import trange, tqdm

model.to(device)

# 外层进度条：设置 position=0
outer_bar = trange(epochs, desc="Training Epochs", position=0, leave=True, dynamic_ncols=True)
for epoch in outer_bar:
    epoch_loss = 0.0
    # 内层进度条：设置 position=1，这样内层的更新会固定在外层进度条下方
    inner_bar = tqdm(loader, desc="Batches", position=1, leave=True, dynamic_ncols=True)
    for batch_data in inner_bar:
        optimizer.zero_grad()                           # 清空梯度
        out = model(batch_data)                         # 前向传播
        loss = criterion(out, batch_data.y.view(-1))      # 计算MSE损失
        loss.backward()                                 # 反向传播计算梯度
        optimizer.step()                                # 更新参数
        epoch_loss += loss.item()
        # 每次迭代时更新内层进度条的附加信息
        inner_bar.set_postfix({"Batch Loss": f"{loss.item():.4f}"})
    # 更新外层进度条的附加信息
    outer_bar.set_postfix({"Epoch Loss": f"{epoch_loss:.4f}"})
    inner_bar.close()  # 关闭当前epoch的内层进度条
outer_bar.close()

Training Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

Batches:   0%|          | 0/10000 [00:00<?, ?it/s]

Batches:   0%|          | 0/10000 [00:00<?, ?it/s]

Batches:   0%|          | 0/10000 [00:00<?, ?it/s]

Batches:   0%|          | 0/10000 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/10000 [00:00<?, ?it/s]

Batches:   0%|          | 0/10000 [00:00<?, ?it/s]

Batches:   0%|          | 0/10000 [00:00<?, ?it/s]

Batches:   0%|          | 0/10000 [00:00<?, ?it/s]

Batches:   0%|          | 0/10000 [00:00<?, ?it/s]

In [11]:
torch.cuda.is_available()

True